<a href="https://colab.research.google.com/github/Bentley97/NLU_First_assignment/blob/main/FirstAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import spacy

test_sentence = "I saw a man with a telescope"

**FUNCTION 1**

In [33]:
def extract_dependency_path(sentence):
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(sentence)

  paths = []

  for token in doc:
    token_path = []

    while token != token.head:  #we can use also (token != token.sent.root)
      token_path.insert(0, token.text)
      #token_path.insert(0, "goes with dep '"+token.dep_+"' in")
      token_path.insert(0, "--"+token.dep_+"->")
      token = token.head

    token_path.insert(0, token.text)
    #token_path.insert(0, "goes with dep '"+token.dep_+"' in")
    token_path.insert(0, "--"+token.dep_+"->")
    token_path.insert(0, "ROOT")
    paths.append(token_path)
  
  return paths


In [34]:
### TEST ###
dep_paths = extract_dependency_path(test_sentece)

for p in dep_paths:
  print(p)

['ROOT', '--ROOT->', 'saw', '--nsubj->', 'I']
['ROOT', '--ROOT->', 'saw']
['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--det->', 'a']
['ROOT', '--ROOT->', 'saw', '--dobj->', 'man']
['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with']
['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope', '--det->', 'a']
['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope']


**FUNCTION 2**

In [22]:
import spacy

test_sentece = "I saw a man with a telescope"

nlp = spacy.load('en_core_web_sm')
doc = nlp(test_sentece)

print(test_sentece)

#for sent in doc.sents:
    #for token in sent:
        #print("{}\t{}\t{}\t{}\t{}".format(token.i, token.text, token.head, token.dep_, sent.))
        
for token in doc:
  print("{}\t{}\t{}\t{}\t{}".format(token.i, token.text, token.head, token.dep_, token))


I saw a man with a telescope
0	I	saw	nsubj	I
1	saw	saw	ROOT	saw
2	a	man	det	a
3	man	saw	dobj	man
4	with	man	prep	with
5	a	telescope	det	a
6	telescope	with	pobj	telescope
